In [1]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import h5py

C:\Users\neal\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
# x = np.random.randn(100,769) # simulated chess data
# y = np.random.randint(0,2,100) # random board state
def getTest(whitewins, blackwins):
    test = []
    test_l = []
    for i in range(whitewins.shape[0]):
        if np.random.randn(1)>0.5:
            test.append([whitewins[i],blackwins[i]])
            test_l.append([1, 0])
        else:
            test.append([blackwins[i],whitewins[i]])
            test_l.append([0, 1])
#         temp_test = np.zeros((len(test), 2, input_size))
#     for i in range(len(test)):
#         first = bitifyFEN(test[i][0])	
#         second = bitifyFEN(test[i][1])	
#         elem = [first,second]
#         temp_test[i] = elem
    return (np.array(test), np.array(test_l))

def getBatchAE(start, size):
    global whiteWins 
    global blackWins 
    size = size//2
    start = start*size

    temp1 = np.concatenate((whiteWins[start:start+size],blackWins[start:start+size]),axis=0)
    np.random.shuffle(temp1)
    return temp1

# ff = h5py.File("raw_chess_data.hdf5", "r")
with h5py.File("raw_chess_data.hdf5", 'r') as ff:
# print(ff.keys())
    whiteWins2 = ff['white'][:]
    blackWins2 = ff['black'][:]
# validation_test, validation_test_l = getTest(whiteWins2[:100000],blackWins2[:100000])
print(whiteWins2.shape)
print(blackWins2.shape)

(2598560, 769)
(1925000, 769)


In [3]:
def getBatchMLP(start, size):
    import random
    global whiteWins 
    global blackWins 

    xR = []
    lR = []

    for i in range(start,start+size):
        if random.random() > 0.5:
            elem = [whiteWins[i], blackWins[i]]
            elem_l = [1,0]
        else:
            elem = [blackWins[i], whiteWins[i]]
            elem_l = [0,1]
        xR.append(elem)
        lR.append(elem_l)
    return (np.array(xR), np.array(lR))

In [4]:
# create initial filters to train individually:
# print(y)
whiteWins = np.random.permutation(whiteWins2[100000:])[:100000]
blackWins = np.random.permutation(blackWins2[100000:])[:100000]

In [43]:
# make whitewins and blackwins
# whitewins = x[np.where(y==1)[0]]
# blackwins = x[np.where(y==0)[0]]

# def shuffle(n):
# #     ind1 = np.random.randint(0,whitewins.shape[0],n)
# #     ind2 = np.random.randint(0,blackwins.shape[0],n)
#     ind1 = np.random.randint(0,whiteWins2.shape[0],n)
#     ind2 = np.random.randint(0,blackWins2.shape[0],n)
    
#     x = []
#     y = []
#     for i in range(n):
#         chance = np.random.rand()
#         if chance>0.5:
#             x.append([whiteWins2[ind1[i]], blackWins2[ind2[i]]])
#             y.append([1,0])
#         else:
#             x.append([blackWins2[ind2[i]], whiteWins2[ind1[i]]])
#             y.append([0,1])
# #     x = [whitewins[ind1], bl]
#     x = np.array(x)
# #     print(x.shape)
#     y = np.array(y)
#     return [x[:,0],x[:,1]], y
# shuffle(30)

def shuffle(n):
#     ind1 = np.random.randint(0,whitewins.shape[0],n)
#     ind2 = np.random.randint(0,blackwins.shape[0],n)
    ind1 = sorted(np.random.randint(0,whiteWins2.shape[0],n))
    ind2 = sorted(np.random.randint(0,blackWins2.shape[0],n))
    
    x1 = np.random.permutation(whiteWins2[ind1])
    x2 = np.random.permutation(blackWins2[ind2])
    
#     print(x1.shape, x2.shape)
#     print(whiteWins2.shape)
#     return
    
    
    x = []
    y = []
    for i in range(n):
        chance = np.random.rand()
        if chance>0.5:
            x.append([x1[i], x2[i]])
            y.append([1,0])
        else:
            x.append([x2[i], x1[i]])
            y.append([0,1])
#     x = [whitewins[ind1], bl]
    x = np.array(x)
#     print(x.shape)
    y = np.array(y)
    return [x[:,0],x[:,1]], y
# print(whiteWins2)

In [40]:
# %%timeit 
a, b = shuffle(500)
print(a[1:10][0].shape)

(500, 2, 769)
(500, 769)


In [ ]:
def train_manual_autoencoder(x, ae_epochs=20):
    from keras.layers import Input, Dense
    from keras.models import Model, Sequential
    from keras import backend as K
    
    a1 = 600
    a2 = 400
    a3 = 200
    a4 = 100
    
    model = Sequential()
    model.add(Dense(a1, input_shape=(x.shape[1],)))
    model.add(Dense(a2))
    model.add(Dense(a3))
    model.add(Dense(a4))
    model.add(Dense(x.shape[1]))
    
    model.compile(optimizer='adam',
                  loss='mse', metrics=['mse'])
    #train the model
    model.fit(x, x, epochs=ae_epochs, 
                  batch_size = 256,
                  verbose=1)
    return model

train_manual_autoencoder(np.concatenate((blackWins, whiteWins),axis=0), 150)
    

Epoch 1/150
200000/200000 [==============================] - 4s 21us/step - loss: 0.0085 - mean_squared_error: 0.0085
Epoch 2/150
200000/200000 [==============================] - 4s 19us/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 3/150
200000/200000 [==============================] - 4s 19us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 4/150
200000/200000 [==============================] - 4s 20us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 5/150
200000/200000 [==============================] - 4s 19us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 6/150
200000/200000 [==============================] - 4s 19us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 7/150
200000/200000 [==============================] - 4s 20us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 8/150
200000/200000 [==============================] - 4s 21us/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 9/150
200000/200000 [=============================

In [10]:
def train_autoencoder(x, ae_epochs=20):
    
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras import backend as K
    
    a1 = 600
    a2 = 400
    a3 = 200
    a4 = 100
    
    pairs = [[x.shape[1],a1],[a1,a2],[a2,a3],[a3,a4]]
    layer_weights = []
    print(pairs)
    
#     inputs = Input(shape=(x.shape[1],))
    for i in pairs:
        # train model
        inputs = Input(shape=(i[0],))
        l1 = Dense(i[1], activation = 'elu')(inputs)
        out = Dense(i[0], activation = 'elu')(l1)
        model = Model(inputs=inputs, outputs=out)
        model.compile(optimizer='adam',
                  loss='mse', metrics=['mse'])
        model.fit(x, x, epochs=ae_epochs, 
                  batch_size = 256,
                  verbose=1)
#         model.evaluate(x, x)
        
        # extract weights and output intermediate values:
        layer_weights.append(model.layers[1].get_weights())
        inp = model.input                                           # input placeholder
        outputs = [layer.output for layer in model.layers]# all layer outputs
        functor = K.function([inp]+ [K.learning_phase()], [outputs[1]] ) # evaluation function
        x = functor([x, 1.])[0]
        print(x.shape)
        
        
#     print(layer_weights)
    # reconstruct the model using the weights:
    inputs = Input(shape = (pairs[0][0],))
    ae1 = Dense(a1, activation='elu',trainable=False)(inputs)
    ae2 = Dense(a2, activation='elu',trainable=False)(ae1)
    ae3 = Dense(a3, activation='elu',trainable=False)(ae2)
    out = Dense(a4, activation='elu',trainable=False)(ae3)
    ae = Model(inputs, out)
    count = 0
    for layer in ae.layers:
        if count>0:
            layer.set_weights(layer_weights[count-1])
    # need to assign weights - maybe compile the model and save it?
#     return inputs, ae1, ae2, ae3, out, layer_weights
    return ae
#     return layer_weights

ae = train_autoencoder(np.concatenate((blackWins, whiteWins),axis=0), 3)
ae.save('autoencoder.h5')

[[769, 600], [600, 400], [400, 200], [200, 100]]
Epoch 1/3
200000/200000 [==============================] - 4s 18us/step - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 2/3
200000/200000 [==============================] - 3s 16us/step - loss: 2.2089e-04 - mean_squared_error: 2.2089e-04
Epoch 3/3
200000/200000 [==============================] - 3s 17us/step - loss: 1.9449e-04 - mean_squared_error: 1.9449e-04
(200000, 600)
Epoch 1/3
200000/200000 [==============================] - 3s 15us/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 2/3
200000/200000 [==============================] - 3s 14us/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 3/3
200000/200000 [==============================] - 3s 14us/step - loss: 0.0012 - mean_squared_error: 0.0012
(200000, 400)
Epoch 1/3
200000/200000 [==============================] - 3s 15us/step - loss: 0.0086 - mean_squared_error: 0.0086
Epoch 2/3
200000/200000 [==============================] - 3s 14us/step - loss: 0.0039 - mean_s

AttributeError: 'Model' object has no attribute 'save_model'

In [29]:
# ae.save('autoencoder.h5')
ae = keras.models.load_model('autoencoder.h5')

C:\Users\neal\Anaconda3\envs\keras\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [46]:
d1 = 400
d2 = 200
d3 = 100
d4 = 2

# build 2 autoencoders
a1 = 600
a2 = 400
a3 = 200
a4 = 100

# input1 = Input(shape = (x.shape[1],))
# ae1 = Dense(a1, activation='relu',trainable=False)(input1)
# ae2 = Dense(a2, activation='relu',trainable=False)(ae1)
# ae3 = Dense(a3, activation='relu',trainable=False)(ae2)
# out1 = Dense(a4, activation='relu',trainable=False)(ae3)

# input2 = Input(shape = (x.shape[1],))
# ae1 = Dense(a1, activation='relu',trainable=False)(input2)
# ae2 = Dense(a2, activation='relu',trainable=False)(ae1)
# ae3 = Dense(a3, activation='relu',trainable=False)(ae2)
# out2 = Dense(a4, activation='relu',trainable=False)(ae3)

# input1 = Input(shape=(x.shape[1],))
# input2 = Input(shape=(x.shape[1],))

input1 = Input(shape=(769,))
input2 = Input(shape=(769,))

batch_size=2560

out1 = ae(input1)
out2 = ae(input2)

l1 = keras.layers.Concatenate()([out1, out2])
l2 = Dense(d1, activation='elu')(l1)
l3 = Dense(d2, activation='elu')(l2)
l4 = Dense(d3, activation='elu')(l3)
out = Dense(d4, activation='softmax')(l4)

model = Model([input1, input2], out)
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

def myGenerator():
    while 1:
        x, y = shuffle(1000000)
        for i in range(1000000//batch_size):
            yield [[x[0][i*batch_size:(i+1)*batch_size], x[1][i*batch_size:(i+1)*batch_size]],
                   y[i*batch_size:(i+1)*batch_size,:]]

hist = model.fit_generator(myGenerator(), 
                    steps_per_epoch = 1000000//batch_size, 
                    epochs = 250,
                    verbose = 2)
#                     callbacks = [keras.callbacks.EarlyStopping(patience=20)],
#                    validation_data = (xxt, yyt))
# for i in range(1000):
#     xx, y = shuffle(500)
#     print(i)
#     model.fit(xx, y, epochs=i, initial_epoch=i-1, verbose=0)

Epoch 1/250
 - 21s - loss: 0.2350 - acc: 0.5954
Epoch 2/250
 - 19s - loss: 0.2285 - acc: 0.6161
Epoch 3/250
 - 18s - loss: 0.2249 - acc: 0.6271
Epoch 4/250
 - 18s - loss: 0.2223 - acc: 0.6335
Epoch 5/250
 - 19s - loss: 0.2210 - acc: 0.6371
Epoch 6/250
 - 20s - loss: 0.2198 - acc: 0.6400
Epoch 7/250
 - 18s - loss: 0.2187 - acc: 0.6422
Epoch 8/250
 - 18s - loss: 0.2184 - acc: 0.6430
Epoch 9/250
 - 19s - loss: 0.2172 - acc: 0.6460
Epoch 10/250
 - 18s - loss: 0.2165 - acc: 0.6478
Epoch 11/250
 - 18s - loss: 0.2158 - acc: 0.6501
Epoch 12/250
 - 18s - loss: 0.2152 - acc: 0.6511
Epoch 13/250
 - 19s - loss: 0.2145 - acc: 0.6527
Epoch 14/250
 - 20s - loss: 0.2140 - acc: 0.6542
Epoch 15/250
 - 20s - loss: 0.2133 - acc: 0.6556
Epoch 16/250
 - 19s - loss: 0.2125 - acc: 0.6578
Epoch 17/250
 - 20s - loss: 0.2125 - acc: 0.6573
Epoch 18/250
 - 20s - loss: 0.2117 - acc: 0.6586
Epoch 19/250
 - 20s - loss: 0.2114 - acc: 0.6602
Epoch 20/250
 - 21s - loss: 0.2112 - acc: 0.6605
Epoch 21/250
 - 22s - loss: 0

In [90]:
import h5py
x1 = np.random.rand(200000,769)
x2 = np.random.rand(200000,769) # about 2gb of ram
with h5py.File('chess_data.hdf5','w') as f:
    dset = f.create_dataset("x1", x1.shape, chunks=(2500, x1.shape[1]), compression="gzip")
    dset[:] = x1
    dset = f.create_dataset("x2", x2.shape, chunks=(2500, x2.shape[1]), compression="gzip")
    dset[:] = x2

In [11]:
# reset memory:
x1 = 0
x2 = 0
# randomly shuffle the data, getting a random permutation, but without needing to hold the array twice:
def low_mem_shuffle(n, filename):
    # n number of random samples. Maybe it will automatically load chunks if I just randomly permute the data:
    out = []
    with h5py.File(filename+'.hdf5','r') as f:
        x1 = f['white'][:]
        x2 = f['black'][:]
        ind1 = sorted(np.random.permutation(range(x1.shape[0]))[:n])
        ind2 = sorted(np.random.permutation(range(x2.shape[0]))[:n])
        xx1 = np.random.permutation(x1[ind1])
        xx2 = np.random.permutation(x2[ind2])
    x = []
    y = []
    for i in range(xx1.shape[0]):
        chance = np.random.rand()
        if chance > 0.5:
            x.append([xx1[i], xx2[i]])
            y.append([0,1])
        else:
            x.append([xx2[i], xx1[i]])
            y.append([1,0])
    x = np.array(x)
    y = np.array(y)
    return [x[:,0], x[:,1]], y
a, b = low_mem_shuffle(50000,'raw_chess_data')
#         out.append

In [12]:
%%timeit
a, b = low_mem_shuffle(50000,'raw_chess_data')

KeyboardInterrupt: 

In [128]:
print(b[:,0].mean())

0.498
